In [1]:
# Import our dependencies

import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy import inspect
from sqlalchemy import text
import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
file_path = "/content/drive/MyDrive/Colab Notebooks/male_fertility.sqlite"
# #################################################
# # Database Setup
# #################################################
# # create engine to hawaii.sqlite
engine = create_engine(f"sqlite:///{file_path}")
insp = inspect(engine)
print(insp.get_table_names())
# # reflect an existing database into a new model
base = automap_base()
# # # # reflect the tables
# base.prepare(autoload_with=engine)
# base.classes.keys()
session = Session(bind=engine)

['male_sperm']


In [3]:
with engine.connect() as connection:
    result = connection.execute(text("SELECT * FROM male_sperm"))
    for row in result:
        print(row)


(1, 2, 1, 1, 1, 1, 1, 6, 1, 4, 7, 3)
(2, 2, 1, 1, 1, 2, 1, 6, 1, 3, 4, 1)
(3, 2, 1, 1, 1, 2, 1, 6, 1, 5, 7, 1)
(4, 3, 1, 1, 2, 1, 2, 5, 1, 5, 10, 1)
(5, 2, 1, 1, 1, 1, 1, 6, 1, 3, 5, 1)
(6, 2, 1, 1, 1, 2, 1, 7, 1, 4, 5, 2)
(7, 2, 1, 1, 1, 1, 1, 6, 1, 5, 4, 3)
(8, 2, 1, 1, 1, 2, 1, 7, 1, 4, 1, 5)
(9, 3, 1, 1, 2, 3, 1, 6, 1, 5, 1, 5)
(10, 3, 1, 1, 1, 2, 1, 5, 1, 5, 5, 3)
(11, 2, 1, 1, 1, 2, 1, 7, 1, 4, 7, 7)
(12, 1, 1, 1, 1, 1, 1, 6, 1, 5, 2, 1)
(13, 1, 1, 1, 1, 1, 1, 6, 1, 4, 1, 3)
(14, 2, 3, 3, 1, 2, 1, 5, 1, 4, 6, 1)
(15, 2, 1, 1, 1, 1, 1, 7, 1, 4, 6, 1)
(16, 3, 1, 1, 1, 2, 1, 5, 1, 5, 6, 3)
(17, 2, 1, 1, 1, 2, 1, 6, 1, 5, 3, 1)
(18, 2, 1, 1, 1, 2, 2, 7, 1, 4, 8, 3)
(19, 3, 1, 1, 1, 3, 1, 7, 1, 5, 8, 1)
(20, 2, 1, 1, 1, 2, 1, 3, 1, 5, 2, 2)
(21, 2, 1, 1, 1, 2, 2, 6, 1, 4, 4, 3)
(22, 3, 1, 1, 1, 3, 2, 7, 1, 5, 9, 1)
(23, 3, 1, 1, 1, 3, 2, 7, 1, 5, 5, 1)
(24, 2, 1, 1, 1, 1, 1, 5, 1, 5, 4, 2)
(25, 2, 1, 1, 1, 1, 1, 7, 1, 4, 3, 1)
(26, 2, 1, 1, 1, 1, 1, 6, 1, 4, 3, 1)
(27, 1, 1, 1, 1, 1, 

In [4]:
# Define your SQL query
query = text("SELECT * FROM male_sperm")

# Execute the query and load data into a DataFrame
with engine.connect() as connection:
    result = connection.execute(query)
    # Fetch all rows into a list of tuples
    rows = result.fetchall()

# Get the column names from the result set
columns = result.keys()

# Create a DataFrame from the rows and columns
male_sperm = pd.DataFrame(rows, columns=columns)

# Print the DataFrame
print(male_sperm)






      ID  Age  Religion  Ethnicity  Maritalstatus  Marriageperiod  \
0      1    2         1          1              1               1   
1      2    2         1          1              1               2   
2      3    2         1          1              1               2   
3      4    3         1          1              2               1   
4      5    2         1          1              1               1   
..   ...  ...       ...        ...            ...             ...   
304  305    4         1          1              2               2   
305  306    2         1          1              1               2   
306  307    2         1          1              1               3   
307  308    3         1          1              2               1   
308  309    2         1          1              1               2   

     No.ofchildren  Educationallevel  Employmentstatus  Workingperiod  \
0                1                 6                 1              4   
1                1       

In [5]:
male_sperm = male_sperm.drop(columns = ['Religion'], axis =1)
male_sperm

,ID,Age,Ethnicity,Maritalstatus,Marriageperiod,No.ofchildren,Educationallevel,Employmentstatus,Workingperiod,Householdincome,Spermquality
0,1,2,1,1,1,1,6,1,4,7,3
1,2,2,1,1,2,1,6,1,3,4,1
2,3,2,1,1,2,1,6,1,5,7,1
3,4,3,1,2,1,2,5,1,5,10,1
4,5,2,1,1,1,1,6,1,3,5,1
...,...,...,...,...,...,...,...,...,...,...,...
304,305,4,1,2,2,2,6,1,5,7,3
305,306,2,1,1,2,2,9,1,5,8,1
306,307,2,1,1,3,1,6,1,5,2,1
307,308,3,1,2,1,3,5,1,5,2,1


In [6]:
male_sperm = pd.get_dummies(male_sperm, dtype=float)
male_sperm.head()

,ID,Age,Ethnicity,Maritalstatus,Marriageperiod,No.ofchildren,Educationallevel,Employmentstatus,Workingperiod,Householdincome,Spermquality
0,1,2,1,1,1,1,6,1,4,7,3
1,2,2,1,1,2,1,6,1,3,4,1
2,3,2,1,1,2,1,6,1,5,7,1
3,4,3,1,2,1,2,5,1,5,10,1
4,5,2,1,1,1,1,6,1,3,5,1


In [7]:
countsbinning = male_sperm['Householdincome'].value_counts()
countsbinning

5     46
4     39
6     38
2     35
3     31
7     28
1     26
8     25
9     23
10    18
Name: Householdincome, dtype: int64

In [8]:
y = male_sperm['No.ofchildren'].values
y


array([1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2,
       2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 2, 1,
       1, 2, 1, 1, 1, 3, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 2, 1,
       1, 1, 1, 1, 2, 1, 1, 3, 2, 1, 1, 3, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1,
       1, 2, 1, 2, 2, 1, 3, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 3, 1, 2,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 2, 1,
       3, 2, 2, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1,
       1, 2, 1, 1, 2, 2, 3, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1,
       1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 3, 1, 1, 2, 1, 1, 1, 3, 1,
       1, 2, 2, 1, 1, 1, 1, 1, 2, 2, 1, 3, 2, 1, 2, 1, 1, 2, 1, 3, 2, 1,
       1, 1, 1, 2, 3, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 2,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1,
       2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1,

In [9]:
X = male_sperm.drop('No.ofchildren', axis =1).values
X

array([[  1,   2,   1, ...,   4,   7,   3],
       [  2,   2,   1, ...,   3,   4,   1],
       [  3,   2,   1, ...,   5,   7,   1],
       ...,
       [307,   2,   1, ...,   5,   2,   1],
       [308,   3,   1, ...,   5,   2,   1],
       [309,   2,   1, ...,   5,   6,   1]])

In [10]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 42)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
le= LabelEncoder().fit(male_sperm['Spermquality'])
male_sperm['Spermquality'] = le.transform(male_sperm['Spermquality'])
male_sperm['Spermquality'].value_counts()

0    144
1     52
2     48
7     31
4     24
6      8
3      1
5      1
Name: Spermquality, dtype: int64

In [13]:
male_sperm.columns

Index(['ID', 'Age', 'Ethnicity', 'Maritalstatus', 'Marriageperiod',
       'No.ofchildren', 'Educationallevel', 'Employmentstatus',
       'Workingperiod', 'Householdincome', 'Spermquality'],
      dtype='object')

In [14]:
X= male_sperm[['Age', 'Maritalstatus',
       'No.ofchildren', 'Educationallevel', 'Employmentstatus',
       'Workingperiod', 'Householdincome']]

y= male_sperm['Spermquality']

In [15]:
# Train the model
model = tf.keras.models.Sequential([tf.keras.layers.Dense(256, activation ='relu', input_shape=(7,)),
                                    tf.keras.layers.Dense(128, activation ='relu'),
                                    tf.keras.layers.Dense(10, activation='softmax')])

In [16]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
# Evaluate the model using the test data
model.fit(X, y, epochs=50, validation_split=0.15)

Epoch 1/50
9/9 [==============================] - 2s 60ms/step - loss: 1.8637 - accuracy: 0.4122 - val_loss: 2.3348 - val_accuracy: 0.1915
Epoch 2/50
9/9 [==============================] - 0s 17ms/step - loss: 1.4839 - accuracy: 0.5153 - val_loss: 2.1957 - val_accuracy: 0.1915
Epoch 3/50
9/9 [==============================] - 0s 12ms/step - loss: 1.4298 - accuracy: 0.5153 - val_loss: 2.1380 - val_accuracy: 0.1915
Epoch 4/50
9/9 [==============================] - 0s 13ms/step - loss: 1.4501 - accuracy: 0.5153 - val_loss: 1.9740 - val_accuracy: 0.1915
Epoch 5/50
9/9 [==============================] - 0s 19ms/step - loss: 1.4193 - accuracy: 0.5153 - val_loss: 2.1864 - val_accuracy: 0.1915
Epoch 6/50
9/9 [==============================] - 0s 12ms/step - loss: 1.3995 - accuracy: 0.5153 - val_loss: 2.0211 - val_accuracy: 0.1915
Epoch 7/50
9/9 [==============================] - 0s 17ms/step - loss: 1.4039 - accuracy: 0.5153 - val_loss: 2.0857 - val_accuracy: 0.1915
Epoch 8/50
9/9 [===========

In [18]:
model_loss, model_accuracy = model.evaluate(X,y,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

10/10 - 0s - loss: 1.3791 - accuracy: 0.4854 - 157ms/epoch - 16ms/step
Loss: 1.3790631294250488, Accuracy: 0.48543688654899597


In [19]:
#Optimization1
# Use less columns

X= male_sperm[['Age', 'Maritalstatus',
       'No.ofchildren', 'Educationallevel', 'Employmentstatus',
       'Workingperiod']]

y= male_sperm['Spermquality']


In [20]:
# Train the model
model = tf.keras.models.Sequential([tf.keras.layers.Dense(512, activation ='relu', input_shape=(6,)),
                                    tf.keras.layers.Dense(256, activation ='relu'),
                                    tf.keras.layers.Dense(128, activation ='relu'),
                                    tf.keras.layers.Dense(10, activation='softmax')])


model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
# Evaluate the model using the test data
model.fit(X, y, epochs=50, validation_split=.3)

Epoch 1/50
7/7 [==============================] - 2s 66ms/step - loss: 1.7336 - accuracy: 0.5046 - val_loss: 2.7069 - val_accuracy: 0.2043
Epoch 2/50
7/7 [==============================] - 0s 18ms/step - loss: 1.3616 - accuracy: 0.5787 - val_loss: 2.4118 - val_accuracy: 0.2043
Epoch 3/50
7/7 [==============================] - 0s 18ms/step - loss: 1.2624 - accuracy: 0.5787 - val_loss: 2.2398 - val_accuracy: 0.2043
Epoch 4/50
7/7 [==============================] - 0s 18ms/step - loss: 1.2434 - accuracy: 0.5787 - val_loss: 2.2495 - val_accuracy: 0.2043
Epoch 5/50
7/7 [==============================] - 0s 10ms/step - loss: 1.2362 - accuracy: 0.5787 - val_loss: 2.3543 - val_accuracy: 0.2043
Epoch 6/50
7/7 [==============================] - 0s 12ms/step - loss: 1.2456 - accuracy: 0.5787 - val_loss: 2.4303 - val_accuracy: 0.2043
Epoch 7/50
7/7 [==============================] - 0s 11ms/step - loss: 1.2377 - accuracy: 0.5787 - val_loss: 2.2320 - val_accuracy: 0.2043
Epoch 8/50
7/7 [===========

In [22]:
model_loss, model_accuracy = model.evaluate(X,y,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

10/10 - 0s - loss: 1.4962 - accuracy: 0.4984 - 39ms/epoch - 4ms/step
Loss: 1.4962347745895386, Accuracy: 0.4983818829059601


In [23]:
#Further optimization increase epochs

# Train the model
model = tf.keras.models.Sequential([tf.keras.layers.Dense(512, activation ='relu', input_shape=(6,)),
                                    tf.keras.layers.Dense(256, activation ='relu'),
                                    tf.keras.layers.Dense(128, activation ='relu'),
                                    tf.keras.layers.Dense(10, activation='softmax')])


model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [24]:
model.fit(X, y, epochs=100, validation_split=.15)

Epoch 1/100
9/9 [==============================] - 1s 38ms/step - loss: 1.8215 - accuracy: 0.5115 - val_loss: 2.2326 - val_accuracy: 0.1915
Epoch 2/100
9/9 [==============================] - 0s 11ms/step - loss: 1.4796 - accuracy: 0.5153 - val_loss: 2.0997 - val_accuracy: 0.1915
Epoch 3/100
9/9 [==============================] - 0s 9ms/step - loss: 1.4473 - accuracy: 0.5153 - val_loss: 2.0921 - val_accuracy: 0.1915
Epoch 4/100
9/9 [==============================] - 0s 8ms/step - loss: 1.4249 - accuracy: 0.5153 - val_loss: 2.0717 - val_accuracy: 0.1915
Epoch 5/100
9/9 [==============================] - 0s 9ms/step - loss: 1.4125 - accuracy: 0.5153 - val_loss: 2.0536 - val_accuracy: 0.1915
Epoch 6/100
9/9 [==============================] - 0s 10ms/step - loss: 1.4162 - accuracy: 0.5153 - val_loss: 2.0921 - val_accuracy: 0.1915
Epoch 7/100
9/9 [==============================] - 0s 9ms/step - loss: 1.4026 - accuracy: 0.5153 - val_loss: 1.9962 - val_accuracy: 0.1915
Epoch 8/100
9/9 [=======

In [25]:
model_loss, model_accuracy = model.evaluate(X,y,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

10/10 - 0s - loss: 1.4023 - accuracy: 0.5437 - 64ms/epoch - 6ms/step
Loss: 1.4023162126541138, Accuracy: 0.5436893105506897


In [26]:
# Optimization 3
#drop more columns
male_sperm = male_sperm.drop(columns = ['ID'], axis =1)
male_sperm

,Age,Ethnicity,Maritalstatus,Marriageperiod,No.ofchildren,Educationallevel,Employmentstatus,Workingperiod,Householdincome,Spermquality
0,2,1,1,1,1,6,1,4,7,2
1,2,1,1,2,1,6,1,3,4,0
2,2,1,1,2,1,6,1,5,7,0
3,3,1,2,1,2,5,1,5,10,0
4,2,1,1,1,1,6,1,3,5,0
...,...,...,...,...,...,...,...,...,...,...
304,4,1,2,2,2,6,1,5,7,2
305,2,1,1,2,2,9,1,5,8,0
306,2,1,1,3,1,6,1,5,2,0
307,3,1,2,1,3,5,1,5,2,0


In [27]:
male_sperm = male_sperm.drop(columns = ['Employmentstatus'], axis =1)
male_sperm

,Age,Ethnicity,Maritalstatus,Marriageperiod,No.ofchildren,Educationallevel,Workingperiod,Householdincome,Spermquality
0,2,1,1,1,1,6,4,7,2
1,2,1,1,2,1,6,3,4,0
2,2,1,1,2,1,6,5,7,0
3,3,1,2,1,2,5,5,10,0
4,2,1,1,1,1,6,3,5,0
...,...,...,...,...,...,...,...,...,...
304,4,1,2,2,2,6,5,7,2
305,2,1,1,2,2,9,5,8,0
306,2,1,1,3,1,6,5,2,0
307,3,1,2,1,3,5,5,2,0


In [28]:
male_sperm.columns

Index(['Age', 'Ethnicity', 'Maritalstatus', 'Marriageperiod', 'No.ofchildren',
       'Educationallevel', 'Workingperiod', 'Householdincome', 'Spermquality'],
      dtype='object')

In [29]:
X= male_sperm[['Age', 'Ethnicity', 'Maritalstatus', 'Marriageperiod', 'No.ofchildren',
       'Educationallevel', 'Workingperiod', 'Householdincome']]

y= male_sperm['Spermquality']

In [30]:
#Further optimization

# Train the model
model = tf.keras.models.Sequential([tf.keras.layers.Dense(512, activation ='relu', input_shape=(8,)),
                                    tf.keras.layers.Dense(256, activation ='relu'),
                                    tf.keras.layers.Dense(128, activation ='relu'),
                                    tf.keras.layers.Dense(10, activation='softmax')])


model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [31]:
model.fit(X, y, epochs=100, validation_split=.15)

Epoch 1/100
9/9 [==============================] - 1s 31ms/step - loss: 1.7141 - accuracy: 0.4542 - val_loss: 2.1293 - val_accuracy: 0.1915
Epoch 2/100
9/9 [==============================] - 0s 10ms/step - loss: 1.4921 - accuracy: 0.5153 - val_loss: 2.0466 - val_accuracy: 0.1915
Epoch 3/100
9/9 [==============================] - 0s 9ms/step - loss: 1.4407 - accuracy: 0.5115 - val_loss: 2.2991 - val_accuracy: 0.1915
Epoch 4/100
9/9 [==============================] - 0s 11ms/step - loss: 1.4431 - accuracy: 0.5153 - val_loss: 2.0002 - val_accuracy: 0.1915
Epoch 5/100
9/9 [==============================] - 0s 10ms/step - loss: 1.3920 - accuracy: 0.5153 - val_loss: 2.1048 - val_accuracy: 0.1915
Epoch 6/100
9/9 [==============================] - 0s 12ms/step - loss: 1.3936 - accuracy: 0.5153 - val_loss: 2.0701 - val_accuracy: 0.1915
Epoch 7/100
9/9 [==============================] - 0s 10ms/step - loss: 1.3981 - accuracy: 0.5115 - val_loss: 2.1879 - val_accuracy: 0.1915
Epoch 8/100
9/9 [====

In [32]:
model_loss, model_accuracy = model.evaluate(X,y,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

10/10 - 0s - loss: 1.0405 - accuracy: 0.6602 - 73ms/epoch - 7ms/step
Loss: 1.0405484437942505, Accuracy: 0.6601941585540771


In [33]:
#Optmization 4 add another layer and increase epochs


# Train the model
model = tf.keras.models.Sequential([tf.keras.layers.Dense(512, activation ='relu', input_shape=(8,)),
                                    tf.keras.layers.Dense(256, activation ='relu'),
                                    tf.keras.layers.Dense(128, activation ='relu'),
                                    tf.keras.layers.Dense(64, activation ='relu'),
                                    tf.keras.layers.Dense(10, activation='softmax')])


model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [34]:
model.fit(X, y, epochs=150, validation_split=.15)

Epoch 1/150
9/9 [==============================] - 2s 54ms/step - loss: 1.7298 - accuracy: 0.4466 - val_loss: 2.5512 - val_accuracy: 0.1915
Epoch 2/150
9/9 [==============================] - 0s 17ms/step - loss: 1.4929 - accuracy: 0.5153 - val_loss: 2.0270 - val_accuracy: 0.1915
Epoch 3/150
9/9 [==============================] - 0s 17ms/step - loss: 1.4518 - accuracy: 0.5153 - val_loss: 2.1464 - val_accuracy: 0.1915
Epoch 4/150
9/9 [==============================] - 0s 16ms/step - loss: 1.4372 - accuracy: 0.5153 - val_loss: 2.0919 - val_accuracy: 0.1915
Epoch 5/150
9/9 [==============================] - 0s 17ms/step - loss: 1.4289 - accuracy: 0.5153 - val_loss: 2.0521 - val_accuracy: 0.1915
Epoch 6/150
9/9 [==============================] - 0s 15ms/step - loss: 1.4071 - accuracy: 0.5153 - val_loss: 2.1916 - val_accuracy: 0.1915
Epoch 7/150
9/9 [==============================] - 0s 17ms/step - loss: 1.4072 - accuracy: 0.5153 - val_loss: 1.9657 - val_accuracy: 0.1915
Epoch 8/150
9/9 [===

In [35]:
model_loss, model_accuracy = model.evaluate(X,y,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

10/10 - 0s - loss: 0.9777 - accuracy: 0.7735 - 40ms/epoch - 4ms/step
Loss: 0.9776732921600342, Accuracy: 0.7734627723693848
